# Ambev data challenge
## Adriano Freitas

## Modelo de previsão de cumprimento da meta

Este notebook tem o objetivo de criar um modelo para prever o cumprimento da meta.

In [ ]:
%%capture

%run ./00-agf-utils.ipynb

%store -r default_color
%store -r default_light_color
%store -r default_dark_color
%store -r colormap
%store -r figsize

In [ ]:
data_path = '../data/processed/'
file_name = 'ambev-final-dataset-processed.csv'

In [ ]:
df = pd.read_csv(os.path.join(data_path, file_name))
df.shape
df.head()

## Definição do target
Baseado no entendimento do dataset, temos 3 tipos de variáveis que retratam o atingimento das metas, são eles:
- **Atingido (ating)**: Qual é o percentual da meta atingido no mês.
- **Pontos (pontos)**: Os pontos são calculados baseado na regra de atingimento parcial (coluna `nom_regra_alcance_parcial`). Este valor é utilizado para calcular o atingimento final da meta.
- **Acumulativo (acum)**: Mostra de forma acumulativa decompondo a meta pelo peso do KPI. Esta coluna é calculada aplicando o peso do kpi sobre os pontos.

Decidimos então prever o percentual da meta que o funcionário vai atingir no mês, uma vez que essa é a medida raiz, sendo as outras colunas calculadas derivadas dessa. Com essa previsão poderemos calcular as demais e chegar na previsão do final do exercício.

## Definição do modelo

Decidimos aplicar uma rede neural recorrente (RNN) com células LSTM (Long Short Term Memory), que possuem ótima performance em séries temporais como essa em questão.

Definimos um intervalo de observação de 3 meses, o que nos dará uma visão de sazonalidade. Nos baseamos na duração de cada estação do ano. Como temos um número de observações diferente a cada mês, vamos usar a média de observações de cada mês multiplicado por 3 como intervalo.

A média encontrada foi de 10621 registros.

In [ ]:
# Definindo quantos registros tem em média por mês
# essa será nossa janela uma vez que cada mês possui uma quantidade diferente
# de registros, isso nos dará aproximadamente 3 meses de observações
# para gerar um previsão
df.groupby(by='ord_mes_referencia').count().mean()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

### Preparando dataframe

#### Scaling e Encoding e filtrando as colunas

Vamos utilizar apenas o funcnioário, o KPI e seu percentual de atingimento.
Precisamos também remover o último mês que será usado para o teste final.

In [ ]:
pattern = re.compile('nom_\w+')
col_search = np.vectorize(lambda x, pattern=pattern: bool(pattern.search(x)))
idx_filter = col_search(df.columns)
nom_cols = df.columns[idx_filter]

In [ ]:
def remove_last_month(df):
    df = df[df['ord_mes_referencia']<12]
    return df

def select_cols(df):
    df = df[['dis_nome_funcionario', 'nom_codigo_kpi', 'per_ating_mes']]
    return df

drop_cols = [
    'ord_mes_referencia', 
    'nom_pais',
    'dis_nome_kpi', 
    'per_peso_kpi', 
    'nom_prazo',
    'nom_cargo',
    'nom_banda',
    'nom_area',
    'nom_regra_alcance_parcial', 
    'bin_meta_projeto', 
    'per_pontos_mes',
    'per_acum_mes', 
    'per_ating_acumulado',
    'per_pontos_acumulado', 
    'per_acum_acumulado', 
    'per_ating_fim_exer',
    'per_pontos_fim_exer', 
    'per_acum_fim_exer'
]

prep_df = Prep(df) \
    .apply_custom(remove_last_month) \
    .apply_custom(select_cols) \
    .encode(['nom_codigo_kpi']) \
    .scale()

In [ ]:
prep_df.df.shape
prep_df.df.info()

#### Separando X e y
Vamos utilizar uma biblioteca de processamento paralelo para preparar o dataset de treino, para isso precisaremos separar os registros previsores (últimos 3 meses) para cada registro do dataset.

In [ ]:
# train_df = prep_df.df.values
# X, y = [], []
# i_len = 10621 * 3

# for i in range (i_len, len(train_df)):
#     X.append(train_df[i - i_len : i, :17])
#     y.append(train_df[i, 17])
    
# X, y = np.array(X), np.array(y)
# print(X.shape, y.shape)

In [ ]:
from dask.distributed import Client, progress
import dask.array as da
from multiprocessing import cpu_count

client = Client(threads_per_worker=4, n_workers=cpu_count())
client

In [ ]:
train_df = da.from_array(prep_df.df.values, chunks=1000)
i_len = 10621 * 3

X = np.array([train_df[:i_len, :2]])
y = np.array([train_df[i_len, 2]])
print('inicio')
X = [delayed(np.append)(X, np.array([train_df[i - i_len : i, :2]]), axis=0) for i in range(i_len+1, len(train_df))]
X = compute(X)
print('fim X')
y = [delayed(np.append)(y, np.array([train_df[i, 2]]), axis=0) for i in range(i_len+1, len(train_df))]
y = compute(y)
print('fim y')

## Criando o modelo

In [ ]:
input_shape = (X.shape[1], 17)

model = Sequential()
model.add(LSTM(units = 100, 
               return_sequences = True, 
               input_shape = input_shape))
model.add(Dropout(0.3))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 50))
model.add(Dropout(0.3))

model.add(Dense(units = 1, activation = 'sigmoid'))  # linear

model.compile(optimizer='rmsprop', 
              loss='mean_squared_error', 
              metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='loss', min_delta=1e-10, patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, verbose=1)
mcp = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                      monitor='loss', save_best_only=True, verbose=1)

model.fit(X, y, epochs=100, batch_size=32, callbacks=[es, reduce_lr, mcp])